In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("customer_shopping_behavior.csv")

In [3]:
df.head(5)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [5]:
df.isnull().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [6]:
# filling review rating column with median
df["Review Rating"] = df.groupby("Category")["Review Rating"].transform(lambda x : x.fillna(x.median()))

In [7]:
df.isnull().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [8]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [9]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [10]:
# create a age_group new column
labels = ["Young adult","Adult", "Middle-Aged", "Seniors"]
df["age_group"] = pd.qcut(df["age"], q=4, labels=labels)

In [11]:
df[["age_group"]].head(5)

,age_group
0,Middle-Aged
1,Young adult
2,Middle-Aged
3,Young adult
4,Middle-Aged


In [12]:
df["frequency_of_purchases"].unique()

array(['Fortnightly', 'Weekly', 'Annually', 'Quarterly', 'Bi-Weekly',
       'Monthly', 'Every 3 Months'], dtype=object)

In [13]:
frequency_mapping = {
    'Fortnightly':14,
    'Weekly':7,
    'Annually':365,
    'Quarterly':90,
    'Bi-Weekly':14,
    'Monthly':30,
    'Every 3 Months':90
}

df["purchase_frequency_days"] = df["frequency_of_purchases"].map(frequency_mapping)

In [14]:
df[["frequency_of_purchases","purchase_frequency_days"]].head(5)

,frequency_of_purchases,purchase_frequency_days
0,Fortnightly,14
1,Fortnightly,14
2,Weekly,7
3,Weekly,7
4,Annually,365


In [15]:
(df['discount_applied'] == df['promo_code_used']).all()

True

In [16]:
df = df.drop('promo_code_used', axis=1)

In [17]:
# !pip install mysql-connector-python

In [18]:
#!pip install mysql-connector-python==8.0.33

In [19]:
#!pip install sqlalchemy pymysql

In [20]:
import mysql.connector

In [21]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="iamroot72",
    database="customer_behaviour",
    port=3306
)

In [22]:
if conn.is_connected():
    print("success")

success


In [23]:
from sqlalchemy import create_engine

In [24]:
engine = create_engine(
    f"mysql+pymysql://{"root"}:{"iamroot72"}@{"localhost"}:{3306}/{"customer_behaviour"}"
)

In [25]:
table_name = "customer"

df.to_sql(
    table_name,
    con=engine,
    if_exists="replace",
    index=False
)

3900